In [1]:
import pydeck as pdk
import pandas as pd
from pandas import DataFrame
from geopy.distance import distance
pd.options.display.max_colwidth = 250

In [2]:
departure_points = [
    {"latitude": 43.06035, "longitude": 35.14627, "elevation": 1},
    # {"latitude": 43.06031, "longitude": 35.15406, "elevation": 10},
    # {"latitude": 43.06044, "longitude": 35.14752, "elevation": 10},
]


vertices_to_explore = [
    {"latitude": 43.09357, "longitude": 35.13925, "elevation": 1000},
    {"latitude": 43.094, "longitude": 35.16036, "elevation": 1000},
    
    {"latitude": 43.10269, "longitude": 35.18285, "elevation": 1000},
    
    {"latitude": 43.09888, "longitude": 35.18114, "elevation": 1000},
    
    {"latitude": 43.09616, "longitude": 35.16774, "elevation": 1000},
    {"latitude": 43.10119, "longitude": 35.14199, "elevation": 1000},
    {"latitude": 43.09918, "longitude": 35.13585, "elevation": 1000},
    {"latitude": 43.10001, "longitude": 35.15152, "elevation": 1000},
    {"latitude": 43.09929, "longitude": 35.14118, "elevation": 1000},
    {"latitude": 43.11903, "longitude": 35.15402, "elevation": 1000},
    {"latitude": 43.12069, "longitude": 35.13669, "elevation": 1000},
    {"latitude": 43.1135, "longitude": 35.1269, "elevation": 1000},
    {"latitude": 43.1243, "longitude": 35.19115, "elevation": 1000},
    
    
    {"latitude": 43.10925, "longitude": 35.16032, "elevation": 1000},
]


In [3]:
df_departure_points = pd.DataFrame(departure_points)
df_departure_points["name"] = df_departure_points.apply(
    lambda row: f"depatrure_{row.name}", axis=1
)
df_departure_points

,latitude,longitude,elevation,name
0,43.06035,35.14627,1,depatrure_0


In [4]:
df_vertices_to_explore = pd.DataFrame(vertices_to_explore)
df_vertices_to_explore["name"] = df_vertices_to_explore.apply(
    lambda row: f"explore_{row.name}", axis=1
)
df_vertices_to_explore

,latitude,longitude,elevation,name
0,43.09357,35.13925,1000,explore_0
1,43.09400,35.16036,1000,explore_1
2,43.10269,35.18285,1000,explore_2
3,43.09888,35.18114,1000,explore_3
4,43.09616,35.16774,1000,explore_4
5,43.10119,35.14199,1000,explore_5
6,43.09918,35.13585,1000,explore_6
7,43.10001,35.15152,1000,explore_7
8,43.09929,35.14118,1000,explore_8
9,43.11903,35.15402,1000,explore_9


In [5]:
from shapely import Polygon, LineString, intersection

view_state = pdk.ViewState(latitude=43.08913, longitude=35.14388, zoom=12)


def swap(polygon):
    result = [(coord[1], coord[0]) for coord in polygon]
    return result


# swap because geoJSON lon first
nogo_zone1 = swap(
    [
        [43.107064367495, 35.15697128055],
        [43.100520570157, 35.159031217074],
        [43.104324685807, 35.167871777987],
        [43.107279055816, 35.159889523958],
    ]
)
nogo_zone1_polygon_without_buffer: Polygon = Polygon(nogo_zone1)

given_from_vertex = [35.15152, 43.10001]
given_to_vertex = [35.16032, 43.10925]
linestring = LineString([given_from_vertex, given_to_vertex])  # long first

departure_points_layer = pdk.Layer(
    "PointCloudLayer",
    data=df_departure_points,
    get_position=["longitude", "latitude", "elevation"],
    get_color=[255, 0, 0, 160],  # RGBA color, here red
    get_radius=100,  # Radius of each point in meters
    pickable=True,
    point_size=10,
    auto_highlight=True,
)


vertices_to_explore_layer = pdk.Layer(
    "PointCloudLayer",
    data=df_vertices_to_explore,
    get_position=["longitude", "latitude", "elevation"],
    get_color=[0, 255, 0, 160],  # RGBA color, here red
    get_radius=100,  # Radius of each point in meters
    pickable=True,
    point_size=8,
    auto_highlight=True,
)

nogo_zone_layer = pdk.Layer(
    "PolygonLayer",
    [nogo_zone1],
    stroked=False,
    get_polygon="-",
    get_elevation=2000,
    get_fill_color=[255, 0, 0, 100],
    extruded=True,
    wireframe=True,
    auto_highlight=True,
    pickable=True,
)

nogo_zone_layer_buffered = pdk.Layer(
    "PolygonLayer",
    [list(nogo_zone1_polygon_without_buffer.exterior.coords)],
    stroked=False,
    get_polygon="-",
    get_elevation=2000,
    get_fill_color=[245, 222, 179, 100],
    extruded=True,
    wireframe=True,
    auto_highlight=True,
    pickable=True,
)

intersects: LineString = intersection(nogo_zone1_polygon_without_buffer, linestring)


point_intersection_a = [
    intersects.coords.xy[0].tolist()[0],
    intersects.coords.xy[1].tolist()[0],
]
point_intersection_b = [
    intersects.coords.xy[0].tolist()[1],
    intersects.coords.xy[1].tolist()[1],
]


intersection_point_with_buffer = pd.DataFrame(
    [
        [
            *point_intersection_a,
            1000,
            "first",
        ],
        [
            *point_intersection_b,
            1000,
            "second",
        ],
    ],
    columns=["longitude", "latitude", "elevation", "name"],
)


intersection_point_with_buffer_layer = pdk.Layer(
    "PointCloudLayer",
    data=intersection_point_with_buffer,
    get_position=["longitude", "latitude", "elevation"],
    get_color=[0, 0, 255, 160],  # RGBA color, here red
    get_radius=200,  # Radius of each point in meters
    pickable=True,
    point_size=6,
    auto_highlight=True,
)

r0 = pdk.Deck(
    layers=[
        departure_points_layer,
        vertices_to_explore_layer,
        intersection_point_with_buffer_layer,
        nogo_zone_layer,
        nogo_zone_layer_buffered,
    ],
    initial_view_state=view_state,
)
r0.show()

In [6]:
from shapely.ops import split
from shapely import GeometryCollection, difference, MultiLineString
from geopy.distance import geodesic

saved_nogo_zones_polylines = {}

def calculate_distance_for_polygon(polygon: Polygon, p_start: list[tuple], p_end: list[float]):
    spliter = LineString([p_start, p_end])
    as_line_string = LineString([*polygon.exterior.coords])
    intersection_original_points_with_line_string = intersection(as_line_string, spliter)
    actual_path = difference(as_line_string, intersection_original_points_with_line_string)
    
    if isinstance(actual_path, MultiLineString):
        print("WAS HERE OMG!!!")
        actual_path = list(actual_path.geoms)[0]
        
    with_start_and_end_points = [p_start, *actual_path.coords, p_end]
    
    distance_polyline = geodesic(
        *swap(with_start_and_end_points)
    ).meters  # swap -> geopy need pair of (lat, lon) tuples
    
    return with_start_and_end_points, int(distance_polyline)


def take_nogo_into_account(p_start: list[float], p_end: list[float]):
    current_line = LineString([p_start, p_end])
    split_result: GeometryCollection = split(
        nogo_zone1_polygon_without_buffer, current_line
    )  # TODO: add array of nogo zones, not just 1

    # no intersection
    if len(split_result.geoms) <= 1:
        return [], -1

    polyline1_after_split, polyline2_after_split = split_result.geoms
    polyline1_after_split, distance_1 = calculate_distance_for_polygon(
        polyline1_after_split, p_start, p_end
    )
    polyline2_after_split, distance_2 = calculate_distance_for_polygon(
        polyline2_after_split, p_start, p_end
    )

    # take shortest path
    if distance_1 < distance_2:
        return polyline1_after_split, distance_1

    return polyline2_after_split, distance_2

In [7]:
def create_distance_matrix(df: DataFrame):
    distance_matrix = pd.DataFrame(index=df["name"], columns=df["name"])
    for _, row1 in df.iterrows():
        for _, row2 in df.iterrows():
            p_start = (row1["longitude"], row1["latitude"])
            p_end = (row2["longitude"], row2["latitude"])
            polyline, distance_in_m = take_nogo_into_account(p_start, p_end)

            if polyline:
                saved_nogo_zones_polylines[f"{p_start}|{p_end}"] = polyline
                distance_matrix.loc[row1["name"], row2["name"]] = int(distance_in_m)
            else:
                dist = distance(
                    (row1["latitude"], row1["longitude"]),
                    (row2["latitude"], row2["longitude"]),
                ).meters
                distance_matrix.loc[row1["name"], row2["name"]] = int(dist)

    return distance_matrix



distance_matrix_meters = create_distance_matrix(
    pd.concat([df_departure_points, df_vertices_to_explore], ignore_index=True)
)

distance_matrix_meters

WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!
WAS HERE OMG!!!


name,depatrure_0,explore_0,explore_1,explore_2,explore_3,explore_4,explore_5,explore_6,explore_7,explore_8,explore_9,explore_10,explore_11,explore_12,explore_13
name,,,,,,,,,,,,,,,
depatrure_0,0,3734,3910,5567,5136,4345,4550,4396,4426,4345,6549,6748,6111,7989,5758
explore_0,3734,0,1719,3691,3461,2337,875,681,1228,654,3073,3020,2431,6483,2466
explore_1,3910,1719,0,2069,1776,646,1695,2076,981,1668,2835,3536,3480,4196,2172
explore_2,5567,3691,2069,0,445,1428,3485,3893,2583,3558,2966,4256,4956,2493,1973
explore_3,5136,3461,1776,445,0,1131,3197,3687,2414,3253,3143,4354,5160,2939,2049
explore_4,4345,2337,646,1428,1131,0,2169,2617,1388,2190,4241,4667,3842,3661,2745
explore_5,4550,875,1695,4088,3197,2169,0,547,786,221,2210,2208,1838,4754,1740
explore_6,4396,681,2076,4092,3687,2617,547,0,1279,434,2655,2390,1749,5295,2284
explore_7,4426,1228,981,2653,2414,1388,786,1279,0,845,2122,2595,2502,5434,1328


In [8]:
import numpy as np

def plot_nogo_zone_polylines(data_structure=saved_nogo_zones_polylines):
    add_layers = []
    for index, value in enumerate(data_structure.values()):
        polyline = [[list((*p, 1000)) for p in value]]
        data = {
            "path": polyline
        }
        df = pd.DataFrame(data)
        layer = pdk.Layer(
            "PathLayer",
            df,
            pickable=True,
            et_width=5,
            get_color=np.random.choice(range(256), size=3).tolist(),  # RGB color format
            width_scale=20,
            width_min_pixels=2,
            get_path="path",  # Column name that contains paths
            get_width=1,
            
        )
        add_layers.append(layer)
        # print(polyline[0][0], polyline[0][1])
    return add_layers

    
nogo_layers = plot_nogo_zone_polylines()

r1 = pdk.Deck(
layers=[
        vertices_to_explore_layer,
        departure_points_layer,
        nogo_zone_layer,
        nogo_zone_layer_buffered,
        *nogo_layers
    ],
    initial_view_state=view_state,
)

r1.show()

In [9]:
from pydantic import BaseModel


class Drone(BaseModel):
    id: str
    fly_capacity_time_minutes: int  # Total time the drone can fly
    speed_kmh: int  # Speed of the drone
    slack_at_vertex_seconds: int  # Time needed at each vertex
    departure_location: list[float]  # Latitude, Longitude, Elevation


drones = [
    Drone(
        id="first",
        fly_capacity_time_minutes=40,
        speed_kmh=30,
        slack_at_vertex_seconds=50,
        departure_location=[43.06035, 35.14627],
    ),
    Drone(
        id="second",
        fly_capacity_time_minutes=20,
        speed_kmh=120,
        slack_at_vertex_seconds=50,
        departure_location=[43.06035, 35.14627],
    ),
    Drone(
        id="third",
        fly_capacity_time_minutes=50,
        speed_kmh=40,
        slack_at_vertex_seconds=50,
        departure_location=[43.06035, 35.14627],
    ),
    # Add more drones as needed
]

In [10]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    data = {
        'distance_matrix': distance_matrix_meters.values.astype(int),  # Ensure distances are integers
        'num_vehicles': len(drones),  # Number of drones
        'depot': 0  # Assuming the first vertex in your list is the depot
    }
    return data

def calculate_max_travel_seconds(drone: Drone, average_stops: int = 4) -> int:
    total_slack_time_seconds = drone.slack_at_vertex_seconds * average_stops
    effective_flying_time_seconds = int(max(0, (drone.fly_capacity_time_minutes * 60) - total_slack_time_seconds))
    return effective_flying_time_seconds

In [11]:
def model():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )
    routing = pywrapcp.RoutingModel(manager)

    def create_callback(drone):
        def travel_time_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            if from_node == to_node:
                return 0
            travel_distance = data["distance_matrix"][from_node][to_node]
            travel_time_seconds = int(travel_distance / (drone.speed_kmh * 1000 / 3600))  # Convert to seconds
            return travel_time_seconds
        return travel_time_callback

    for idx, drone in enumerate(drones):
        slack_time = drone.slack_at_vertex_seconds

        callback = create_callback(drone)
        callback_index = routing.RegisterTransitCallback(callback)
        routing.SetArcCostEvaluatorOfVehicle(callback_index, idx)

        max_travel_time_seconds = calculate_max_travel_seconds(drone, 5)
        routing.AddDimension(
            callback_index,
            slack_time,  # Slack at each vertex in seconds
            max_travel_time_seconds,  # Maximum travel time in seconds
            False,  # Don't start cumul to zero because we have slack
            f"Time_{idx}",
        )
        time_dimension = routing.GetDimensionOrDie(f"Time_{idx}")
        time_dimension.SetGlobalSpanCostCoefficient(100)


    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = 3
    search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution)

    return (data, manager, routing, solution)


def print_solution(data, manager, routing, solution):
    print("Solution:")
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}: "
        while not routing.IsEnd(index):
            plan_output += f"{manager.IndexToNode(index)} -> "
            index = solution.Value(routing.NextVar(index))
        plan_output += f"{manager.IndexToNode(index)}"
        print(plan_output)


data, manager, routing, solution = model()

Solution:
Route for vehicle 0: 0 -> 9 -> 6 -> 2 -> 5 -> 3 -> 4 -> 0
Route for vehicle 1: 0 -> 13 -> 0
Route for vehicle 2: 0 -> 8 -> 14 -> 10 -> 11 -> 12 -> 7 -> 1 -> 0


In [12]:
def get_colors():
    a = 255
    green = [144, 238, 144, a]
    aqua = [0, 255, 255, a]
    wheat = [245, 222, 179, a]
    magenta = [255, 0, 255, a]
    white = [255, 255, 255, a]
    gold = [255, 215, 0, a]
    return [green, aqua, wheat, magenta, white, gold]


def get_routes(data, manager, routing, solution):
    """Extracts the routes from the solution and returns them including the depot as start and end point."""
    routes = []
    for vehicle_id in range(data["num_vehicles"]):
        route = []
        index = routing.Start(vehicle_id)
        route.append(manager.IndexToNode(index))
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                route.append(manager.IndexToNode(index))
        route.append(route[0])
        routes.append(route)
    return routes


def plot_polyline(polyline, color):
    polyline = [list((*p, 1000)) for p in polyline]
    data = {"path": [polyline]}  # Wrap the polyline in a list to make it a single path
    df = pd.DataFrame(data)

    layer = pdk.Layer(
        "PathLayer",
        df,
        pickable=True,
        et_width=5,
        get_color=color[:-1],  # RGB color format
        width_scale=20,
        width_min_pixels=2,
        get_path="path",  # Column name that contains paths
        get_width=3,
    )

    return layer


from copy import deepcopy


def find_matching_values_for_pairs(
    coordinates: list, data_structure=saved_nogo_zones_polylines
):
    result = deepcopy(coordinates)

    for index in range(len(coordinates) - 1):
        for key in data_structure.keys():
            from_coord, to_coord = key.split("|")
            from_coord = eval(from_coord)  # Convert string to tuple
            to_coord = eval(to_coord)  # Convert string to tuple

            if (from_coord == coordinates[index]) and (
                to_coord == coordinates[index + 1]
            ):
                result = (
                    result[: index + 1]
                    + data_structure[key][1:-1]
                    + result[index + 1 :]
                )
                # return data_structure[key]
                print("FOUND A MATCH!")
    return result


def create_route_layers(routes, all_nodes, drones):
    layers = []
    for drone, (route, color) in zip(drones, zip(routes, get_colors())):
        lons = all_nodes.iloc[route]["longitude"].to_list()
        lats = all_nodes.iloc[route]["latitude"].to_list()
        polyline = list(zip(lons, lats))
        polyline_after_match = find_matching_values_for_pairs(polyline)

        layers.append(plot_polyline(polyline_after_match, color))

        data = [
            {
                "longitude": polyline_after_match[1][0],
                "latitude": polyline_after_match[1][1],
                "drone_id": drone.id,
                "altitude": 1000,
            }
        ]
        text_layer = pdk.Layer(
            "TextLayer",
            data,
            get_position=[
                "longitude",
                "latitude",
                "altitude",
            ],  # Position text at the target of each segment
            get_text="drone_id",  # Field containing the text to display
            get_size=24,
            get_color=color,
            get_angle=0,
            getTextAnchor='"middle"',
            get_alignment_baseline='"center"',
        )
        layers.append(text_layer)

    return layers


if solution is None:
    raise BaseException("Couldnt find a solution")

routes = get_routes(data, manager, routing, solution)
all_nodes = pd.concat([df_departure_points, df_vertices_to_explore], ignore_index=True)
route_layers = create_route_layers(routes, all_nodes, drones)

FOUND A MATCH!


In [14]:



r1 = pdk.Deck(
    layers=[
        vertices_to_explore_layer,
        departure_points_layer,
        *route_layers,
        nogo_zone_layer,
        nogo_zone_layer_buffered,
    ],
    initial_view_state=view_state,
)

r1.to_html("mission_plan_result.html")
r1.show()